In [1]:
from manim import *
import numpy as np
import math

Manim Community v0.18.0

In [2]:
config.media_width = "75%"
config.verbosity = "WARNING"

BLACK = ManimColor('#16161d')
FONT = 'Freight Big Pro'

In [3]:
def perpendicular_bisector(a, b, p):
    grad = (a[1]-b[1])/(a[0]-b[0])
    grad_perp = -1/grad
    intercept = a[1] - grad*a[0]
    intercept_perp = p[1] - grad_perp*p[0]
    x = (intercept_perp-intercept)/(grad-grad_perp)
    return [x, grad_perp*x + intercept_perp, 0]

def vector_length(v):
    return (v[0]**2 + v[1]**2)**0.5

In [4]:
%%manim -qp PointSet

class PointSet(MovingCameraScene):
    def construct(self):
        self.P = [(-2,3),(-3,2),(0,2),(2,2),(-1,1),
                  (0,1),(2,1),(-2,0),(3,0),(1,-1),
                  (-2,-2),(-1,-2),(2,-2),(1,-3)]
        self.P = [[p[0],p[1],0] for p in self.P]
        self.simplex = [(1,0),(0,8),(8,13),(13,1)]
    
        self.partitions = [(BLACK,[4,5,9]),(RED,[2,3,6]),(GREEN,[12]),(BLUE,[7,10,11])]
        self.selected_vertex = {1:3,2:12,3:10}

        self.dots = []
        self.lines = []

        self.camera.background_color=BLACK

        self.title = Text('Quickhull:', font_size=32, font=FONT).to_corner(UL)
        self.steps = []
        self.play(Write(self.title))

        # draw all points
        for point in self.P:
            #self.add(Dot(point[0],point[1]))
            d = Dot(point, radius=0.05)
            self.dots.append(d)
            #self.play(Create(d), run_time=0.12)
        self.play(LaggedStart(*[Create(obj) for obj in self.dots],lag_ratio=0.2,run_time=2))
        self.wait()

        # draw bvounding box
        min_x = self.P[self.simplex[0][0]][0]
        max_y = self.P[self.simplex[1][0]][1]
        max_x = self.P[self.simplex[2][0]][0]
        min_y = self.P[self.simplex[3][0]][1]
        corners = [[min_x,min_y,0],[min_x,max_y,0],[max_x,max_y,0],[max_x,min_y,0]]
        self.bounding_box = [
            Text('Bounding Box', font_size=14, font=FONT).move_to([min_x+0.65,max_y+0.2,0]),
            DashedLine(corners[0],corners[1], dash_length=0.05, stroke_width=2),
            DashedLine(corners[1],corners[2], dash_length=0.05, stroke_width=2),
            DashedLine(corners[2],corners[3], dash_length=0.05, stroke_width=2),
            DashedLine(corners[3],corners[0], dash_length=0.05, stroke_width=2)
        ]
        self.add_step('Create Initial Simplex')
        self.play(*[Write(self.bounding_box[0])]+[GrowFromPoint(obj, ORIGIN) for obj in self.bounding_box[1:]])

        # draw simplex

        for edge in self.simplex:
            l = self.connect_indicies(edge[0],edge[1])
            self.lines.append(l)
            self.play(Create(l), run_time=0.5)
        #self.wait()
        self.play(*[FadeOut(obj) for obj in self.bounding_box])
        #self.wait()

        poly = Polygon(*[self.P[self.simplex[i][0]] for i in range(4)], color=WHITE)
        poly.set_z_index(-1)
        poly.set_fill(BLUE, opacity=0.4)
        self.add_step('Discard Inner Points')
        self.play(Create(poly))
        #self.wait()

        # point sweep

        for i in range(len(self.partitions)):
            colour, points = self.partitions[i]
            if colour==BLACK:
                for p in points:
                    self.dots[p].set_z_index(-1)
            else:
                l = self.lines[i].copy()
                l.set(stroke_width=0)
                next_line_direction = self.P[self.selected_vertex[i]] - l.end
                next_line_direction = next_line_direction * vector_length(l.start-l.end)/vector_length(next_line_direction)
                next_line = Line(l.end,l.end+next_line_direction)
                sweep_poly = Polygon(*[l.end, l.start, l.end+next_line_direction], stroke_width=0)
                sweep_poly.set_fill(colour, opacity=0.2)
                sweep_poly.set_z_index(-1)
                self.play(*[FadeTransform(l,next_line), Transform(l.copy(),sweep_poly, replace_mobject_with_target_in_scene=True)], run_time=0.25)
                self.wait(0.5)
                self.play(Uncreate(next_line), Transform(sweep_poly, l), run_time=0.15)

            self.play(*[self.dots[p].animate.set_color(colour) for p in points], lag_ratio=0.1, run_time=0.25)
            if colour==BLACK:
                self.play(Uncreate(poly))
                self.add_step('Partition Remaining Points')
            #self.wait()
        #self.play(Uncreate(t))
        self.wait()

        self.add_step('Select Furthest Points')

        self.camera.frame.save_state()

        # partition 1
        self.play(
            self.camera.frame.animate.set_width(6).move_to(Dot([0.5,1.5,0]))
        )
        #self.wait()

        self.partition_animation(1)

        self.play(Restore(self.camera.frame))

        # partition 2
        self.play(
            self.camera.frame.animate.set_width(6).move_to(Dot([2,-1.5,0]))
        )
        #self.wait()

        self.partition_animation(2)

        self.play(Restore(self.camera.frame))

        # partition 3
        self.play(
            self.camera.frame.animate.set_width(9.5).move_to(Dot([-1,-0.5,0]))
        )
        #self.wait()

        self.partition_animation(3)

        self.play(Restore(self.camera.frame))

        # show all points
        self.play(LaggedStart(*[dot.animate.set_color(WHITE) for dot in self.dots]))
        #self.wait()

        
        self.play(LaggedStart(*[Unwrite(t) for t in self.steps[::-1]+[self.title]],lag_ratio=0.5,run_time=2))
        self.play(LaggedStart(*[Uncreate(obj) for obj in self.lines[len(self.lines):3:-1]+[self.lines[0]]],lag_ratio=0.35,run_time=3))
        self.play(LaggedStart(*[Uncreate(obj) for obj in self.dots],lag_ratio=0.1,run_time=1.5))

    def add_step(self, text):
        self.steps.append(Text('• '+text, font_size=18, font=FONT).to_corner(UL).shift([0,-0.75-len(self.steps)*0.5,0]))
        self.play(Write(self.steps[-1]), run_time=0.75)


    def connect_indicies(self,i,j):
        p_1 = self.P[i]
        p_2 = self.P[j]
        return Line(p_1, p_2)
        #self.lines.append(l)
        #self.play(Create(l), run_time=0.5)
        
    def draw_bisectors(self, n):
        '''draws bisecting lines for partition n'''
        lines = []
        for p_i in self.partitions[n][1]:
            p = self.P[p_i]
            a_i, b_i = self.simplex[n]
            q = perpendicular_bisector(self.P[a_i], self.P[b_i], p)
            new_line = DashedLine(q, p, dash_length=0.05, stroke_width=2)
            new_line.set_z_index(-1)
            lines.append(new_line)
            self.play(Create(new_line), run_time=0.1*len(self.partitions[n][1]))
        return lines
        #anims = [Create(l) for l in lines]#1/(2+len(anims))
        #self.play(LaggedStart(*anims, lag_ratio=0.25, run_time=2))
        #self.play(*[Create(l) for l in lines], lag_ratio=0.5, run_time=0.5)
    
        
    def partition_animation(self, n):
        '''entire animation for partition n'''
        bisectors = self.draw_bisectors(n)
        self.wait()

        anims = [FadeOut(self.lines[n])]
        for line in bisectors:
            anims.append(Uncreate(line))
        a_i, b_i = self.simplex[n]

        l_1 = self.connect_indicies(a_i,self.selected_vertex[n])
        l_1_inverse = self.connect_indicies(self.selected_vertex[n], a_i)
        l_2 = self.connect_indicies(self.selected_vertex[n],b_i)
        for line in [l_1, l_1_inverse, l_2]:
            line.set_z_index(-1)
        anims.append(Create(l_1_inverse))
        self.lines.append(l_1)
        self.lines.append(l_2)
        anims.append(Create(l_2))
        
        for p_i in self.partitions[n][1]:
            if p_i==self.selected_vertex[n]:
                anims.append(self.dots[p_i].animate.set_color(WHITE))
            else:
                anims.append(self.dots[p_i].animate.set_color(BLACK))

        self.play(*anims, run_time=0.75)
        self.add(l_1)
        self.remove(l_1_inverse)

In [5]:
%%manim -qp RecursionRelation

class RecursionRelation(Scene):
    def construct(self):
        self.camera.background_color=BLACK

        ## Best Case        
        LHS = MathTex(r'T(n)').shift(5*LEFT).shift(2.25*UP)
        RHS = [MathTex('=','2',r'T\left(\frac{n}{2}\right)',r'{{ + O(n) }}').shift(2.5*LEFT),
               MathTex('=','2',r'\left[','2',r'{{ T\left(\frac{n}{4}\right) }}','+',r'{{ O\left(\frac{n}{2}\right) }}',r'\right]',r'{{ + O(n) }}'),
               MathTex('=','2',r'\left[','2',r'{{ \left[','2',r'T\left(\frac{n}{8}\right)','+',r'O\left(\frac{n}{4}\right)',r'\right] }}','+',r'{{ O\left(\frac{n}{2}\right) }}',r'\right]',r'{{ + O(n) }}'),
               MathTex('=',r'2^k',r'T\left(\frac{n}{2^k}\right)','+',r'{{ \sum_{i=0}^{k-1} }}',r'2^i O(\frac{n}{2^i})'),# pause 1
               MathTex('=',r'2^k',r'T\left(\frac{n}{2^k}\right)','+',r'{{ \sum_{i=0}^{k-1} }}',r'O(n)'),
               MathTex('=',r'2^k',r'T\left(\frac{n}{2^k}\right)','+',r'{{ \sum_{i=1}^{k} }}',r'O(n)'),# extra pause
               MathTex('=',r'2^k',r'T\left(\frac{n}{2^k}\right)','+','k','O(n)'),# pause 2; note
               MathTex('=','n','T(1)','+','',r'log_2','(n)','','O(n)'),# next line
               MathTex('=',r'{{ n',r'O(1) }}','+','O(',r'log_2','(n)',')','O(n)'),
               MathTex('=',r'{{ O(n) }}','+','O(',r'log_2','(n)',')','O(n)'),
               MathTex('=',r'{{ O(n) }}','+','O(','log','(n)',')','O(n)'),
               MathTex('=',r'{{ O(n) }}','+','O(','n','log','(n)',')'),
               MathTex('=','O(','n','log','(n)',')')]
        for i in range(len(RHS)):
            RHS[i] = RHS[i].align_to(RHS[0],LEFT).shift(2.25*UP)
        #rhs_group = VGroup(*RHS[:6]).arrange([0,0,0], center=False, aligned_edge=LEFT).shift(LEFT).shift(2*UP)
        note = [MathTex(r'\frac{n}{2^k} = 1 \implies n = 2^k', font_size=28),
                MathTex(r'\implies k = log_2(n)', font_size=28)]#, aligned_edge=LEFT
        note_group = VGroup(*note).arrange(DOWN, center=False).to_corner(UR).shift(1.5*LEFT).shift(1.25*DOWN)

        self.title = Text('Best Case :', font_size=38, font=FONT).to_corner(UL).align_to(RHS[0],LEFT)
        self.play(Write(self.title))

        self.play(Write(LHS), run_time=0.2)
        self.play(Write(RHS[0]))
        self.wait(2)
        for i in range(2):
            self.play(TransformMatchingTex(RHS[i],RHS[i+1]))
        self.wait() # pause 1
        self.play(TransformMatchingTex(RHS[2],RHS[3]))
        self.wait()
        for i in range(3):
            self.play(TransformMatchingTex(RHS[3+i],RHS[4+i]))
            self.wait(0.25)
        self.wait() # pause 2
        outline_anim = Create(SurroundingRectangle(note_group,color=WHITE,stroke_width=2,buff=0.2), run_time=0.4)
        self.play(*[Write(note_group, run_time=0.75),outline_anim])
        self.wait()
        
        # next line
        self.play(Write(RHS[7].shift(DOWN)))
        for i in range(2):
            self.play(TransformMatchingTex(RHS[7+i],RHS[8+i].shift(DOWN)))
        for i in range(3):
            self.play(TransformMatchingTex(RHS[9+i],RHS[10+i].shift(DOWN),fade_transform_mismatches=True))
        self.wait()


        ## Worst Case
        LHS = MathTex(r'T(n)').shift(5*LEFT).shift(1*DOWN)
        RHS = [MathTex('=',r'T( {{n-1}} )','+','O(n)').align_to(RHS[0],LEFT),
               MathTex('=',r'T( {{n-2}} )','+','O(n)','+','O(n)'),
               MathTex('=',r'T( {{n-3}} )','+','O(n)','+','O(n)','+','O(n)'),
               MathTex('=',r'T( {{n-k}} )','+',r'\sum_{i=0}^{k-1}','O(n)'),# pause; note
               MathTex('=',r'T( {{ 1 }} )','+',r'\sum_{i=0}^{n}',r'{{ O(n) }}'),# next line
               MathTex('=',r'{{ T(1) }}','+','n',r'O(n)'),
               #MathTex('=',r'{{ T(1) }}','+',r'{{ n',r'O(n) }}','-',r'{{ O(n) }}'),
               MathTex('=',r'{{ O(1) }}','+',r'{{ O(n^2) }}'),
               MathTex('=',r'{{ }}',r'{{ O(n^2) }}')]
        for i in range(len(RHS)):
            RHS[i] = RHS[i].align_to(RHS[0],LEFT).shift(1*DOWN)

        note = MathTex('k=n+1', font_size=28).to_corner(UR).align_to(note_group[0],LEFT).shift(4.5*DOWN)
        #note_group = VGroup(*note).arrange(DOWN, center=False).to_corner(UR).shift(4.5*DOWN)#.align_to(old_note,LEFT)

        self.title = Text('Worst Case :', font_size=38, font=FONT).align_to(RHS[0],LEFT).shift(0.25*UP)
        self.play(Write(self.title))

        self.play(Write(LHS), run_time=0.2)
        self.play(Write(RHS[0]))
        self.wait(0.5)
        for i in range(2):
            self.play(TransformMatchingTex(RHS[i],RHS[i+1]))
        self.wait()
        self.play(TransformMatchingTex(RHS[2],RHS[3]))
        self.wait(0.75) # pause and note
        outline_anim = Create(SurroundingRectangle(note,color=WHITE,stroke_width=2,buff=0.2), run_time=0.2)
        self.play(*[Write(note, run_time=0.5),outline_anim])
        self.wait()
        # next line
        self.play(Write(RHS[4].shift(1.6*DOWN)))
        self.wait()
        for i in range(3):
            self.play(TransformMatchingTex(RHS[4+i],RHS[5+i].shift(1.6*DOWN)))
            self.wait(0.15)
        self.wait()

In [6]:
%%manim -qp ConvexSet

class ConvexSet(Scene):
    def construct(self):
        self.camera.background_color=BLACK

        #angles = [(2*x+1)/6 for x in range(6)]

        self.outer_points = [(3*math.cos(PI*angle), 3*math.sin(PI*angle), 0) for angle in [(2*x+1)/6 for x in range(6)]]
        self.inner_points = [(2*math.cos(PI*angle), 2*math.sin(PI*angle), 0) for angle in [x/6 for x in range(12)]]
        dots = [Dot(p, radius=0.05) for p in self.outer_points]
        
        t = Text('Convex set :', font_size=24, font=FONT).to_corner(UL).shift(DOWN).shift(2*RIGHT)
        self.play(Write(t))
        circle = Circle(radius=3, color=BLACK).set_fill(BLUE_C, opacity=0.75).set_opacity(0.5)
        self.play(GrowFromCenter(circle), run_time=1)
        self.wait()
        # show line segments: green
        lines = [self.create_pair(p_1,p_2) for (p_1, p_2) in [(3,0),(1,10),(11,5),(6,8)]]
        for line in lines:
            self.play(*[Create(x) for x in line], lag_ratio=0.75, run_time=0.5)
        self.wait(0.5)
        for line in lines:
            self.play(*[Uncreate(x) for x in line], lag_ratio=0.75, run_time=0.5)
        # concave
        t_2 = Text('Concave set :', font_size=24, font=FONT).to_corner(UL).shift(DOWN).shift(2*RIGHT)
        inner_circle = Circle(radius=1.5, color=BLACK).set_fill(BLACK, opacity=1)
        self.play(GrowFromCenter(inner_circle),Transform(t,t_2,replace_mobject_with_target_in_scene=True), run_time=1)
        self.wait(0.5)
        ##
        line = Line(self.inner_points[3],self.inner_points[9],color=GREEN_E)
        line_2 = Line([x*0.75 for x in self.inner_points[3]],[x*0.75 for x in self.inner_points[9]],color=RED_E)
        self.play(Create(line),Create(line_2))
        self.wait(0.75)
        self.play(Uncreate(line),Uncreate(line_2))
        self.play(ShrinkToCenter(inner_circle), Unwrite(t_2), run_time=1)
        ##
        t_3 = Text('Convex Hull :', font_size=24, font=FONT).to_corner(UL).shift(DOWN).shift(2*RIGHT)
        #self.play(Write(t_3))
        poly = Polygon(*self.outer_points, color=BLACK)
        poly.set_z_index(-1)
        poly.set_fill(BLUE_C, opacity=0.75)
        anims = [Write(t_3), Transform(circle,poly,replace_mobject_with_target_in_scene=True)] + [Create(d) for d in dots]
        self.play(LaggedStart(*anims, run_time=1.75))
        self.wait()
        self.play(Unwrite(t_3))
        self.play(*[Uncreate(obj) for obj in [poly]+dots], lag_ratio=0.25, run_time=1.5)

    def create_pair(self, p_1, p_2):
        line = Line(self.inner_points[p_1],self.inner_points[p_2], color=GREEN_E)
        dot_1 = Dot(self.inner_points[p_1], color=BLACK, radius=0.05).set_z_index(1)
        dot_2 = Dot(self.inner_points[p_2], color=BLACK, radius=0.05).set_z_index(1)
        return [dot_1, line, dot_2]

In [7]:
%%manim -qp Timeline

class Timeline(Scene):
    def construct(self):
        self.camera.background_color=BLACK

        self.events = [(1977, 'Eddy'),(1978, 'Bykat'),(1990, 'Greenfield'),(1996, 'Barber')]
        self.objs = [self.convert_event_to_obj(e) for e in self.events]

        main_line = Line([-6,0,0],[6,0,0])
        left_dots = [Dot([-6-i*0.15,0,0], radius=0.025) for i in range(3,0,-1)]
        right_dots = [Dot([6+(i+1)*0.15,0,0], radius=0.025) for i in range(1,4)]
        self.play(LaggedStart(*[Create(obj) for obj in left_dots], lag_ratio=0.75, run_time=0.25))
        self.play(Create(main_line), run_time=0.5)
        self.play(LaggedStart(*[Create(obj) for obj in right_dots], lag_ratio=0.75, run_time=0.25))
        
        self.wait(0.5)
        self.draw_index(0)
        self.draw_index(1)
        self.wait(10)
        self.draw_index(2)
        self.wait(2)
        self.draw_index(3)
        self.wait()

        for i in range(4):
            self.undraw_index(i)
        self.play(*[Uncreate(x) for x in right_dots+[main_line]+left_dots])
        self.wait()

    def create_text(self, string, position):
        return Text(string,font_size=18,font=FONT).move_to(position)

    def convert_event_to_obj(self, event):
        width = 11
        year, name = event
        x_ = (year-1977)/19
        x = width*x_ - width/2
        line = Line([x,0.3,0],[x,-0.3,0])
        return [line, self.create_text(name,[x,0.5,0]), self.create_text(str(year),[x,-0.5,0])]

    def draw_index(self, n):
        line, name, year = self.objs[n]
        self.play(*[Create(line), Write(name), Write(year)])
    
    def undraw_index(self, n):
        line, name, year = self.objs[n]
        self.play(*[Uncreate(line), Unwrite(name), Unwrite(year)], run_time=0.25)

In [45]:
%%manim -qp Complexity

class Complexity(Scene):
    def construct(self):
        self.camera.background_color=BLACK

        plot_axes = Axes(x_range=[0, 50, 5], y_range=[0, 100, 10],
                         tips=False, axis_config={"include_numbers": False})
        self.play(Create(plot_axes))
        line_1 = plot_axes.plot(lambda n: n*math.log(n,10), x_range=[1,50], use_smoothing=True, stroke_width=2)
        line_2 = plot_axes.plot(lambda n: n**2, x_range=[0,10], use_smoothing=True, stroke_width=2)
        line_3 = plot_axes.plot(lambda n: n, x_range=[0,50], use_smoothing=True, stroke_width=2)
        self.lines = [line_1, line_2, line_3]
        self.wait()

        delay_times = [1.5,3,5]
        self.complexities = ['O(n log(n))','O(n^2)','O(n)']
        math_obj = [MathTex(string, font_size=48) for string in self.complexities]
        math_obj[0] = math_obj[0].next_to(line_1, UP).align_to(line_1,RIGHT)
        math_obj[1] = math_obj[1].next_to(line_2, RIGHT).align_to(line_2,UP)
        math_obj[2] = math_obj[2].next_to(line_3, UP).align_to(line_3,RIGHT)
        #group = VGroup(*math_obj).arrange(DOWN, buff=1).shift(UP)

        for i in range(3):
            self.play(Create(self.lines[i]), Write(math_obj[i]))
            self.wait(delay_times[i])
        
        self.play(*[Unwrite(t) for t in math_obj]+[Uncreate(l) for l in self.lines], lag_ratio=0.3, run_time=2)
        self.play(Uncreate(plot_axes), run_time=0.5)